In [9]:
import openml
import numpy as np
import pandas as pd
import pickle
import json

In [10]:
path = r'C:\Users\Trogwald\Desktop\push_it\AutoML\meta_learning\util\\'

# 1. Filter flows. Create df_all_flows.pkl and df_sklearn_flows.pkl (DONE)

In [11]:
# download and save
#flows_list = openml.flows.list_flows(output_format='dataframe')
#flows_list.external_version.fillna('', inplace=True)
#flows_list.to_pickle(path+'df_all_flows.pkl')

# then load
flows_list = pd.read_pickle(path+'df_all_flows.pkl')
# 16373 the total number of flows on OpenML
flows_list

,id,full_name,name,version,external_version,uploader
0,1,openml.evaluation.EuclideanDistance(1.0),openml.evaluation.EuclideanDistance,1,,1
1,2,openml.evaluation.PolynomialKernel(1.0),openml.evaluation.PolynomialKernel,1,,1
2,3,openml.evaluation.RBFKernel(1.0),openml.evaluation.RBFKernel,1,,1
3,4,openml.evaluation.area_under_roc_curve(1.0),openml.evaluation.area_under_roc_curve,1,,1
4,5,openml.evaluation.average_cost(1.0),openml.evaluation.average_cost,1,,1
...,...,...,...,...,...,...
16368,19195,sklearn.impute._base.SimpleImputer(35),sklearn.impute._base.SimpleImputer,35,"openml==0.12.2,sklearn==1.1.3",32759
16369,19196,sklearn.tree._classes.DecisionTreeClassifier(30),sklearn.tree._classes.DecisionTreeClassifier,30,"openml==0.12.2,sklearn==1.1.3",32759
16370,19197,sklearn.svm._classes.SVC(12),sklearn.svm._classes.SVC,12,"openml==0.12.2,sklearn==1.1.3",30495
16371,19198,sklearn.neighbors._classification.KNeighborsCl...,sklearn.neighbors._classification.KNeighborsCl...,15,"openml==0.12.2,sklearn==1.1.3",30495


In [12]:
# all flows that related to SKLEARN
sklearn_flows = flows_list[
    # Filter other packages. I need only sklearn
    ~flows_list['external_version'].str.contains('^R_*|^Weka_*|^Moa_*',regex=True,case=True) &
    ~flows_list['name'].str.contains('^torch.nn*',regex=True,case=True) &
    ~flows_list['name'].str.contains('^classif.*',regex=True,case=True) &
    ~flows_list['name'].str.contains('^weka.*',regex=True,case=True) &
    ~flows_list['name'].str.contains('^mlr.*',regex=True,case=True) &

    # Don't need those algorithms
    ~flows_list['name'].str.contains('^sklearn.svm.*classes.NuSVC',regex=True, case=False) &

    # They might be useful if there is no single model. At least you can extract something from them.
    #~flows_list['name'].str.contains('voting.*classifier',regex=True, case=False) &
    #~flows_list['name'].str.contains('VotingClassifier',regex=False, case=False) &
    #~flows_list['name'].str.contains('BaggingClassifier',regex=False, case=False) &
    #~flows_list['name'].str.contains('pipeline',case=False) & # remove Pipelines

    (
    flows_list['name'].str.contains('^sklearn.*',regex=True,case=True) | # remove ?
    flows_list['name'].str.contains('xgboost.sklearn.XGBClassifier',regex=True,case=True) | # remove ?
    flows_list['external_version'].str.contains('.*sklearn.*',regex=True,case=True) # remove ?
    )

]

FLOW_IDS_ALLOWED = sklearn_flows.id.values.tolist()

sklearn_flows.to_pickle(path+'df_sklearn_flows.pkl')
sklearn_flows
# 10193 / 6

#  flags=re.IGNORECASE

,id,full_name,name,version,external_version,uploader
1559,2629,sklearn.ensemble.forest.RandomForestClassifier(8),sklearn.ensemble.forest.RandomForestClassifier,8,Tsklearn_0.17,869
1560,2638,sklearn.ensemble.forest.RandomForestClassifier(9),sklearn.ensemble.forest.RandomForestClassifier,9,Tsklearn_0.16.1,2
1619,2799,sklearn.ensemble.forest.RandomForestClassifier...,sklearn.ensemble.forest.RandomForestClassifier,10,Tsklearn_0.17.1,2
1621,2823,sklearn.ensemble.forest.RandomForestClassifier...,sklearn.ensemble.forest.RandomForestClassifier,11,Tsklearn_0.18.dev0,1140
1622,2856,sklearn.linear_model.logistic.LogisticRegressi...,sklearn.linear_model.logistic.LogisticRegression,1,Tsklearn_0.18.dev0,1140
...,...,...,...,...,...,...
16368,19195,sklearn.impute._base.SimpleImputer(35),sklearn.impute._base.SimpleImputer,35,"openml==0.12.2,sklearn==1.1.3",32759
16369,19196,sklearn.tree._classes.DecisionTreeClassifier(30),sklearn.tree._classes.DecisionTreeClassifier,30,"openml==0.12.2,sklearn==1.1.3",32759
16370,19197,sklearn.svm._classes.SVC(12),sklearn.svm._classes.SVC,12,"openml==0.12.2,sklearn==1.1.3",30495
16371,19198,sklearn.neighbors._classification.KNeighborsCl...,sklearn.neighbors._classification.KNeighborsCl...,15,"openml==0.12.2,sklearn==1.1.3",30495


### further filtering of evals (by model)

In [13]:
#~sklearn_flows['name'].str.contains('SVC',regex=False,case=False)

svm_flows = sklearn_flows[
    sklearn_flows['name'].str.contains('AdaBoostClassifier',regex=False,case=False)
]
svm_flows
#randomforest_flows.to_pickle(path+'df_randomforest_all_flows.pkl')


,id,full_name,name,version,external_version,uploader
1662,3414,sklearn.ensemble.weight_boosting.AdaBoostClass...,sklearn.ensemble.weight_boosting.AdaBoostClass...,1,Tsklearn_0.17,869
1663,3415,sklearn.ensemble.weight_boosting.AdaBoostClass...,sklearn.ensemble.weight_boosting.AdaBoostClass...,2,Tsklearn_0.17.1,901
1684,3439,sklearn.ensemble.weight_boosting.AdaBoostClass...,sklearn.ensemble.weight_boosting.AdaBoostClass...,3,Tsklearn_0.18.dev0,1035
1860,3717,325b3936TESTSENTINEL999sklearn.pipeline.Pipeli...,325b3936TESTSENTINEL999sklearn.pipeline.Pipeli...,1,sklearn_0.18.dev0,2
1866,3723,325b3936TESTSENTINEL999sklearn.ensemble.weight...,325b3936TESTSENTINEL999sklearn.ensemble.weight...,1,sklearn_0.18.dev0,2
...,...,...,...,...,...,...
16168,18972,sklearn.pipeline.Pipeline(columntransformer=sk...,sklearn.pipeline.Pipeline(columntransformer=sk...,3,"openml==0.12.2,sklearn==0.20.0",6691
16169,18973,sklearn.ensemble.weight_boosting.AdaBoostClass...,sklearn.ensemble.weight_boosting.AdaBoostClass...,16,"openml==0.12.2,sklearn==0.20.0",6691
16341,19168,sklearn.pipeline.Pipeline(numerical=sklearn.pi...,sklearn.pipeline.Pipeline(numerical=sklearn.pi...,1,"openml==0.12.2,sklearn==1.0.2",32117
16342,19169,sklearn.ensemble._weight_boosting.AdaBoostClas...,sklearn.ensemble._weight_boosting.AdaBoostClas...,3,"openml==0.12.2,sklearn==1.0.2",32117


In [554]:
sklearn_flows[
        sklearn_flows['full_name'].str.contains('MLP',regex=True,case=False) #|
        #sklearn_flows['full_name'].str.contains('^sklearn.svm._classes.SVC',regex=True,case=False)
    ]

,id,full_name,name,version,external_version,uploader
1671,3425,sklearn.neural_network.multilayer_perceptron.M...,sklearn.neural_network.multilayer_perceptron.M...,1,Tsklearn_0.18.dev0,1035
3375,5867,sklearn.pipeline.Pipeline(Imputer=openml.utils...,sklearn.pipeline.Pipeline(Imputer=openml.utils...,1,"openml==0.2.1,sklearn==0.18",1
3376,5868,sklearn.model_selection._search.RandomizedSear...,sklearn.model_selection._search.RandomizedSear...,1,sklearn==0.18,1
3377,5869,sklearn.neural_network.multilayer_perceptron.M...,sklearn.neural_network.multilayer_perceptron.M...,2,sklearn==0.18,1
4042,6538,sklearn.pipeline.Pipeline(standardscaler=sklea...,sklearn.pipeline.Pipeline(standardscaler=sklea...,1,sklearn==0.18.1,1104
...,...,...,...,...,...,...
16176,18980,sklearn.pipeline.Pipeline(columntransformer=sk...,sklearn.pipeline.Pipeline(columntransformer=sk...,2,"openml==0.12.2,sklearn==0.20.0",6691
16177,18981,sklearn.neural_network.multilayer_perceptron.M...,sklearn.neural_network.multilayer_perceptron.M...,20,"openml==0.12.2,sklearn==0.20.0",6691
16349,19176,sklearn.pipeline.Pipeline(numerical=sklearn.pi...,sklearn.pipeline.Pipeline(numerical=sklearn.pi...,1,"openml==0.12.2,sklearn==1.0.2",32117
16350,19177,sklearn.neural_network._multilayer_perceptron....,sklearn.neural_network._multilayer_perceptron....,1,"openml==0.12.2,sklearn==1.0.2",32117


In [99]:
# 'sklearn.svm.classes.SVC' or 'sklearn.svm._classes.SVC'
# 'sklearn.ensemble.forest.RandomForestClassifier' or 'sklearn.ensemble._forest.RandomForestClassifier'
# 'sklearn.ensemble.weight_boosting.AdaBoostClassifier' or sklearn.ensemble._weight_boosting.AdaBoostClassifier
# 'sklearn.neural_network.multilayer_perceptron.MLPClassifier' or 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'
'sklearn.tree.tree.DecisionTreeClassifier'
'sklearn.ensemble.forest.ExtraTreesClassifier'

'sklearn.linear_model.logistic.LogisticRegression'
'sklearn.svm.classes.LinearSVC'

'sklearn.neural_network.multilayer_perceptron.MLPClassifier'

'sklearn.neighbors.classification.KNeighborsClassifier'
'xgboost.sklearn.XGBClassifier'

'BaggingClassifier'



# filtered algorithms
'sklearn.svm.classes.NuSVC'

# evaluations_setups filtering

In [369]:
evals_tid_1 = openml.evaluations.list_evaluations_setups(
    function='predictive_accuracy', # predictive_accuracy is mostly used in study1
    tasks=[2],
    sort_order = 'desc',
    output_format = 'dataframe'
)
evals_tid_1

,run_id,task_id,setup_id,flow_id,flow_name,data_id,data_name,function,upload_time,uploader,uploader_name,value,values,array_data,parameters
0,8056,2,115,77,weka.LogitBoost_DecisionStump(1),2,anneal,predictive_accuracy,2014-05-22 01:53:10,2,Joaquin Vanschoren,0.996659,None,None,"{'weka.LogitBoost_DecisionStump(1)_F': '0', 'w..."
1,8309,2,116,77,weka.LogitBoost_DecisionStump(1),2,anneal,predictive_accuracy,2014-05-22 02:12:50,2,Joaquin Vanschoren,0.996659,None,None,"{'weka.LogitBoost_DecisionStump(1)_F': '0', 'w..."
2,158766,2,961,471,weka.LogitBoost_DecisionStump(2),2,anneal,predictive_accuracy,2015-02-12 08:19:51,2,Joaquin Vanschoren,0.996659,None,None,"{'weka.LogitBoost_DecisionStump(2)_E': '1', 'w..."
3,256258,2,2043,1191,weka.LogitBoost_DecisionStump(3),2,anneal,predictive_accuracy,2015-03-05 05:01:58,2,Joaquin Vanschoren,0.996659,None,None,"{'weka.LogitBoost_DecisionStump(3)_P': '100', ..."
4,286557,2,2044,1191,weka.LogitBoost_DecisionStump(3),2,anneal,predictive_accuracy,2015-03-12 18:33:13,2,Joaquin Vanschoren,0.996659,None,None,"{'weka.LogitBoost_DecisionStump(3)_P': '100', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5515,178080,2,1844,1074,weka.OLM(3),2,anneal,predictive_accuracy,2015-02-23 11:17:31,1,Jan van Rijn,0.329621,None,None,"{'weka.OLM(3)_R': '0', 'weka.OLM(3)_C': '1', '..."
5516,574159,2,2893,2257,weka.OLM(4),2,anneal,predictive_accuracy,2016-05-31 19:51:54,1,Jan van Rijn,0.329621,None,None,"{'weka.OLM(4)_R': '0', 'weka.OLM(4)_C': '1', '..."
5517,578187,2,3714,2791,weka.KStar(4),2,anneal,predictive_accuracy,2016-06-03 16:07:47,1,Jan van Rijn,0.008909,None,None,"{'weka.KStar(4)_B': '20', 'weka.KStar(4)_M': 'a'}"
5518,10477,2,52,130,weka.KStar(1),2,anneal,predictive_accuracy,2014-05-22 06:24:25,2,Joaquin Vanschoren,0.008909,None,None,"{'weka.KStar(1)_B': '20', 'weka.KStar(1)_M': 'a'}"


In [398]:
# if evals_tid_1.empty == False:
sklearn_evaluations_setups = evals_tid_1.loc[evals_tid_1['flow_id'].isin(FLOW_IDS_ALLOWED)].reset_index()
sklearn_evaluations_setups

,index,run_id,task_id,setup_id,flow_id,flow_name,data_id,data_name,function,upload_time,uploader,uploader_name,value,values,array_data,parameters
0,37,10160806,2,8086270,9689,sklearn.pipeline.Pipeline(imputer=sklearn.prep...,2,anneal,predictive_accuracy,2019-04-13 06:09:08,8420,T I,0.991091,None,None,{'sklearn.pipeline.Pipeline(imputer=sklearn.pr...
1,91,1860456,2,36565,5909,sklearn.pipeline.Pipeline(imputer=sklearn.prep...,2,anneal,predictive_accuracy,2017-03-27 22:32:43,1022,Randal Olson,0.983296,None,None,{'sklearn.ensemble.forest.RandomForestClassifi...
2,175,5950015,2,3984760,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-14 15:39:47,1,Jan van Rijn,0.963252,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...
3,176,5950070,2,3984815,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-14 15:43:34,1,Jan van Rijn,0.962138,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...
4,177,4855681,2,2891090,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-08 06:43:16,1,Jan van Rijn,0.961024,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3601,5489,5930403,2,3965152,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-14 01:15:43,1,Jan van Rijn,0.685969,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...
3602,5490,5956713,2,3991454,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-14 23:11:06,1,Jan van Rijn,0.685969,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...
3603,5491,6043409,2,4078067,6952,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-24 23:10:11,1,Jan van Rijn,0.672606,None,None,{'openmlstudy14.preprocessing.ConditionalImput...
3604,5492,4970024,2,3005421,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-08 19:09:37,1,Jan van Rijn,0.669265,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...


In [403]:
sklearn_evaluations_setups.parameters.values[0]


{'sklearn.pipeline.Pipeline(imputer=sklearn.preprocessing.imputation.Imputer,estimator=sklearn.tree.tree.DecisionTreeClassifier)(15)_memory': 'null',
 'sklearn.pipeline.Pipeline(imputer=sklearn.preprocessing.imputation.Imputer,estimator=sklearn.tree.tree.DecisionTreeClassifier)(15)_steps': '[{"oml-python:serialized_object": "component_reference", "value": {"step_name": "imputer", "key": "imputer"}}, {"oml-python:serialized_object": "component_reference", "value": {"step_name": "estimator", "key": "estimator"}}]',
 'sklearn.preprocessing.imputation.Imputer(41)_axis': '0',
 'sklearn.preprocessing.imputation.Imputer(41)_copy': 'true',
 'sklearn.preprocessing.imputation.Imputer(41)_missing_values': '"NaN"',
 'sklearn.preprocessing.imputation.Imputer(41)_strategy': '"mean"',
 'sklearn.preprocessing.imputation.Imputer(41)_verbose': '0',
 'sklearn.tree.tree.DecisionTreeClassifier(39)_class_weight': 'null',
 'sklearn.tree.tree.DecisionTreeClassifier(39)_criterion': '"gini"',
 'sklearn.tree.tre

In [ ]:
'sklearn.ensemble.forest.RandomForestClassifier'
'sklearn.tree.tree.DecisionTreeClassifier'

'sklearn.svm.classes.SVC'


# no
'sklearn.ensemble.weight_boosting.AdaBoostClassifier'


# 2. Filter Setups. Don't need this (DONE)

In [437]:
flow_id = 18002
setup_for_flow_id = openml.setups.list_setups(flow=flow_id, output_format='dataframe')
setup_for_flow_id

,setup_id,flow_id,parameters
8268584,8268584,18002,"{121507: {'input_id': 121507, 'flow_id': 17654..."
8268694,8268694,18002,"{121507: {'input_id': 121507, 'flow_id': 17654..."
8268781,8268781,18002,"{121507: {'input_id': 121507, 'flow_id': 17654..."
8268800,8268800,18002,"{121507: {'input_id': 121507, 'flow_id': 17654..."
8269111,8269111,18002,"{121507: {'input_id': 121507, 'flow_id': 17654..."
8269252,8269252,18002,"{121507: {'input_id': 121507, 'flow_id': 17654..."
8269552,8269552,18002,"{121507: {'input_id': 121507, 'flow_id': 17654..."
8269566,8269566,18002,"{121507: {'input_id': 121507, 'flow_id': 17654..."
8271928,8271928,18002,"{121507: {'input_id': 121507, 'flow_id': 17654..."
8271952,8271952,18002,"{121507: {'input_id': 121507, 'flow_id': 17654..."


In [439]:
df_params = pd.DataFrame(setup_for_flow_id.values[0,2])
df_params.T
# df_params = pd.DataFrame.from_records(setup_for_flow_id.values[:,2])
# df_params

#setup_for_flow_id.values[:,2]


,input_id,flow_id,flow_name,full_name,parameter_name,data_type,default_value,value
121507,121507,17654,sklearn.ensemble._forest.RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifie...,bootstrap,boolean,true,true
121508,121508,17654,sklearn.ensemble._forest.RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifie...,ccp_alpha,non,0.0,0.8505066338246248
121509,121509,17654,sklearn.ensemble._forest.RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifie...,class_weight,dict,null,null
121510,121510,17654,sklearn.ensemble._forest.RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifie...,criterion,string,"""gini""","""entropy"""
121511,121511,17654,sklearn.ensemble._forest.RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifie...,max_depth,integer or None,null,3
121512,121512,17654,sklearn.ensemble._forest.RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifie...,max_features,int,"""auto""",1
121513,121513,17654,sklearn.ensemble._forest.RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifie...,max_leaf_nodes,int or None,null,404
121514,121514,17654,sklearn.ensemble._forest.RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifie...,max_samples,int or float,null,0.09789220601566292
121515,121515,17654,sklearn.ensemble._forest.RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifie...,min_impurity_decrease,float,1e-07,0.522677637983528
121516,121516,17654,sklearn.ensemble._forest.RandomForestClassifier,sklearn.ensemble._forest.RandomForestClassifie...,min_impurity_split,float,0,null


# Don't need this (DONE)

In [293]:
test_setup1 = openml.setups.get_setup(5578)
test_setup1.parameters

{25569: OpenML Parameter
 ID............: 25569
 Flow ID.......: 3413
 Flow Name.....: sklearn.svm.classes.SVC(1)_kernel
 Flow URL......: https://www.openml.org/f/3413
 Parameter Name: kernel
   |__Data Type: None
   |__Default..: None
   |__Value....: rbf,
 25570: OpenML Parameter
 ID............: 25570
 Flow ID.......: 3413
 Flow Name.....: sklearn.svm.classes.SVC(1)_C
 Flow URL......: https://www.openml.org/f/3413
 Parameter Name: C
   |__Data Type: None
   |__Default..: None
   |__Value....: 3.0517578125e-05,
 25571: OpenML Parameter
 ID............: 25571
 Flow ID.......: 3413
 Flow Name.....: sklearn.svm.classes.SVC(1)_verbose
 Flow URL......: https://www.openml.org/f/3413
 Parameter Name: verbose
   |__Data Type: None
   |__Default..: None
   |__Value....: False,
 25572: OpenML Parameter
 ID............: 25572
 Flow ID.......: 3413
 Flow Name.....: sklearn.svm.classes.SVC(1)_probability
 Flow URL......: https://www.openml.org/f/3413
 Parameter Name: probability
   |__Data Type: 

In [294]:
test_setup2 = openml.setups.get_setup(5580)
test_setup2.parameters

{25569: OpenML Parameter
 ID............: 25569
 Flow ID.......: 3413
 Flow Name.....: sklearn.svm.classes.SVC(1)_kernel
 Flow URL......: https://www.openml.org/f/3413
 Parameter Name: kernel
   |__Data Type: None
   |__Default..: None
   |__Value....: rbf,
 25570: OpenML Parameter
 ID............: 25570
 Flow ID.......: 3413
 Flow Name.....: sklearn.svm.classes.SVC(1)_C
 Flow URL......: https://www.openml.org/f/3413
 Parameter Name: C
   |__Data Type: None
   |__Default..: None
   |__Value....: 211.904839651,
 25571: OpenML Parameter
 ID............: 25571
 Flow ID.......: 3413
 Flow Name.....: sklearn.svm.classes.SVC(1)_verbose
 Flow URL......: https://www.openml.org/f/3413
 Parameter Name: verbose
   |__Data Type: None
   |__Default..: None
   |__Value....: False,
 25572: OpenML Parameter
 ID............: 25572
 Flow ID.......: 3413
 Flow Name.....: sklearn.svm.classes.SVC(1)_probability
 Flow URL......: https://www.openml.org/f/3413
 Parameter Name: probability
   |__Data Type: Non

# 4. Create ALLOWED_TASKS.pkl and did_tid.pkl (DONE)
I narrowed the number of tasks, by restriction (Dataset_id in Study1.datasets)

In [285]:
df_tasks = openml.tasks.list_tasks(
    task_type=openml.tasks.TaskType.SUPERVISED_CLASSIFICATION,
    output_format='dataframe'
)

# unique values
# ttid [TaskType.SUPERVISED_CLASSIFICATION]
# task_type [Supervised Classification]
# estimation_procedure [11 different types (CV, holdout, ...)]
# evaluation_measures [38 different types, some are useless (run_cpu_time, ...)]
df_tasks

,tid,ttid,did,name,task_type,status,estimation_procedure,evaluation_measures,source_data,target_feature,...,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures,cost_matrix
2,2,TaskType.SUPERVISED_CLASSIFICATION,2,anneal,Supervised Classification,active,10-fold Crossvalidation,predictive_accuracy,2,class,...,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0,NaN
3,3,TaskType.SUPERVISED_CLASSIFICATION,3,kr-vs-kp,Supervised Classification,active,10-fold Crossvalidation,NaN,3,class,...,3.0,1527.0,2.0,37.0,3196.0,0.0,0.0,0.0,37.0,NaN
4,4,TaskType.SUPERVISED_CLASSIFICATION,4,labor,Supervised Classification,active,10-fold Crossvalidation,predictive_accuracy,4,class,...,3.0,20.0,2.0,17.0,57.0,56.0,326.0,8.0,9.0,NaN
5,5,TaskType.SUPERVISED_CLASSIFICATION,5,arrhythmia,Supervised Classification,active,10-fold Crossvalidation,predictive_accuracy,5,class,...,13.0,2.0,13.0,280.0,452.0,384.0,408.0,206.0,74.0,NaN
6,6,TaskType.SUPERVISED_CLASSIFICATION,6,letter,Supervised Classification,active,10-fold Crossvalidation,NaN,6,class,...,26.0,734.0,26.0,17.0,20000.0,0.0,0.0,16.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361141,361141,TaskType.SUPERVISED_CLASSIFICATION,128,"BNG(cylinder-bands,nominal,1000000)",Supervised Classification,active,10-fold Crossvalidation,area_under_roc_curve,128,ink_color,...,429.0,422977.0,2.0,40.0,1000000.0,0.0,0.0,0.0,40.0,NaN
361142,361142,TaskType.SUPERVISED_CLASSIFICATION,128,"BNG(cylinder-bands,nominal,1000000)",Supervised Classification,active,10-fold Crossvalidation,predictive_accuracy,128,press_type,...,429.0,422977.0,2.0,40.0,1000000.0,0.0,0.0,0.0,40.0,NaN
361143,361143,TaskType.SUPERVISED_CLASSIFICATION,10,lymph,Supervised Classification,active,10-fold Crossvalidation,NaN,10,class,...,8.0,2.0,4.0,19.0,148.0,0.0,0.0,3.0,16.0,NaN
361144,361144,TaskType.SUPERVISED_CLASSIFICATION,44344,Iris,Supervised Classification,active,10-fold Crossvalidation,NaN,44344,class,...,NaN,50.0,3.0,5.0,150.0,0.0,0.0,4.0,1.0,NaN


In [283]:
study = openml.study.get_study(study_id=1)
DATASET_ID_ALLOWED = study.data

In [374]:
path = r"C:\Users\Trogwald\Desktop\push_it\AutoML\meta_learning\util\\"

df_allowed_tasks = df_tasks.loc[df_tasks['did'].isin(DATASET_ID_ALLOWED)].sort_values(by=['did']).reset_index().fillna('')

df_allowed_tasks = df_allowed_tasks[
    ~df_allowed_tasks['evaluation_measures'].str.contains('run_cpu_time') &
    ~df_allowed_tasks['evaluation_measures'].str.contains('confusion_matrix') &
    ~df_allowed_tasks['evaluation_measures'].str.contains('mean_absolute_error') &
    ~df_allowed_tasks['evaluation_measures'].str.contains('root_mean_squared_error') &
    ~df_allowed_tasks['evaluation_measures'].str.contains('c_index')
]

# save
df_allowed_tasks.to_pickle(path+'df_allowed_tasks.pkl')

# save
ALLOWED_TASKS = df_allowed_tasks.tid.values.tolist()
with open(path+r'ALLOWED_TASKS.pkl', 'wb') as f:
    pickle.dump(ALLOWED_TASKS, f)

# save
did_tid = df_allowed_tasks.groupby('did')['tid'].apply(list).to_dict()
with open(path+r'did_tid.pkl', 'wb') as f:
    pickle.dump(did_tid, f)


df_allowed_tasks

,index,tid,ttid,did,name,task_type,status,estimation_procedure,evaluation_measures,source_data,...,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures,cost_matrix
0,2,2,TaskType.SUPERVISED_CLASSIFICATION,2,anneal,Supervised Classification,active,10-fold Crossvalidation,predictive_accuracy,2,...,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0,
1,1766,1766,TaskType.SUPERVISED_CLASSIFICATION,2,anneal,Supervised Classification,active,5 times 2-fold Crossvalidation,predictive_accuracy,2,...,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0,
2,1882,1882,TaskType.SUPERVISED_CLASSIFICATION,2,anneal,Supervised Classification,active,10 times 10-fold Crossvalidation,predictive_accuracy,2,...,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0,
3,1945,1945,TaskType.SUPERVISED_CLASSIFICATION,2,anneal,Supervised Classification,active,Leave one out,predictive_accuracy,2,...,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0,
4,7286,7286,TaskType.SUPERVISED_CLASSIFICATION,2,anneal,Supervised Classification,active,10-fold Crossvalidation,,2,...,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0,"[[0,1,2]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,3954,3954,TaskType.SUPERVISED_CLASSIFICATION,1120,MagicTelescope,Supervised Classification,active,10-fold Crossvalidation,,1120,...,2.0,6688.0,2.0,12.0,19020.0,0.0,0.0,11.0,1.0,
1614,4660,4660,TaskType.SUPERVISED_CLASSIFICATION,1121,badges2,Supervised Classification,active,10 times 10-fold Crossvalidation,predictive_accuracy,1121,...,2.0,84.0,2.0,11.0,294.0,0.0,0.0,7.0,4.0,
1615,146084,146084,TaskType.SUPERVISED_CLASSIFICATION,1121,badges2,Supervised Classification,active,10-fold Crossvalidation,precision,1121,...,2.0,84.0,2.0,11.0,294.0,0.0,0.0,7.0,4.0,
1616,167004,167004,TaskType.SUPERVISED_CLASSIFICATION,1121,badges2,Supervised Classification,active,33% Holdout set,,1121,...,2.0,84.0,2.0,11.0,294.0,0.0,0.0,7.0,4.0,


In [542]:
pd.read_pickle(r'C:\Users\Trogwald\Desktop\push_it\AutoML\meta_learning\parameters_results_study1\803\3668.pkl')

,run_id,task_id,setup_id,flow_id,flow_name,data_id,data_name,function,upload_time,uploader,uploader_name,value,values,array_data,parameters
0,29163,3668,769,523,weka.Bagging_J48(2),803,delta_ailerons,predictive_accuracy,2014-10-11 02:26:03,2,Joaquin Vanschoren,0.945434,None,None,"{'weka.Bagging_J48(2)_I': '40', 'weka.Bagging_..."
1,257869,3668,2054,1194,weka.RandomSubSpace_REPTree(4),803,delta_ailerons,predictive_accuracy,2015-03-05 08:51:27,2,Joaquin Vanschoren,0.945294,None,None,"{'weka.REPTree(9)_M': '2', 'weka.REPTree(9)_V'..."
2,75403,3668,808,532,weka.Bagging_RandomForest(2),803,delta_ailerons,predictive_accuracy,2014-12-01 06:45:44,2,Joaquin Vanschoren,0.945013,None,None,"{'weka.Bagging_RandomForest(2)_I': '10', 'weka..."
3,32073,3668,771,523,weka.Bagging_J48(2),803,delta_ailerons,predictive_accuracy,2014-10-13 17:00:54,2,Joaquin Vanschoren,0.944873,None,None,"{'weka.Bagging_J48(2)_I': '160', 'weka.Bagging..."
4,27428,3668,768,523,weka.Bagging_J48(2),803,delta_ailerons,predictive_accuracy,2014-10-08 15:25:01,2,Joaquin Vanschoren,0.944733,None,None,"{'weka.Bagging_J48(2)_I': '20', 'weka.Bagging_..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,88249,3668,1573,397,weka.BayesianLogisticRegression(2),803,delta_ailerons,predictive_accuracy,2014-12-03 02:23:47,2,Joaquin Vanschoren,0.469351,None,None,{'weka.BayesianLogisticRegression(2)_D': 'true...
559,88251,3668,1573,397,weka.BayesianLogisticRegression(2),803,delta_ailerons,predictive_accuracy,2014-12-03 02:23:49,2,Joaquin Vanschoren,0.469351,None,None,{'weka.BayesianLogisticRegression(2)_D': 'true...
560,106052,3668,609,396,weka.AdaBoostM1_BayesianLogisticRegression(2),803,delta_ailerons,predictive_accuracy,2014-12-09 00:12:02,2,Joaquin Vanschoren,0.469351,None,None,{'weka.AdaBoostM1_BayesianLogisticRegression(2...
561,144957,3668,609,396,weka.AdaBoostM1_BayesianLogisticRegression(2),803,delta_ailerons,predictive_accuracy,2015-01-31 09:29:29,2,Joaquin Vanschoren,0.469351,None,None,{'weka.AdaBoostM1_BayesianLogisticRegression(2...


# Create df_dids_for_train.pkl, df_dids_for_test.pkl (DONE)
Some tasks were filtered out

In [4]:
import os
import pandas as pd

all_dids = pd.read_csv('C:\\Users\\Trogwald\\Desktop\\push_it\\AutoML\\meta_learning\\datasets_metafeatures\\study1_metafeatures.csv').Dataset_ID.values.tolist()
all_dids = set(all_dids)

did_tid = pd.read_pickle('C:\\Users\\Trogwald\\Desktop\\push_it\\AutoML\\meta_learning\\util\\did_tid.pkl')
task_exist = []
task_not_exist = []
dataset_exist = []


for did, tid_list in did_tid.items():
    for tid in tid_list:
        file_path = r'C:\Users\Trogwald\Desktop\push_it\AutoML\meta_learning\parameters_results_study1\\' +\
          str(did) + '\\' + str(tid) + '.pkl'

        is_exist = os.path.exists(file_path)
        if is_exist:
            task_exist.append(tid)
            dataset_exist.append(did)
        else:
            task_not_exist.append(tid)

# remove duplicates
dataset_exist = set(dataset_exist)

# datasets from Study1 that were excluded
dataset_not_exist = all_dids - dataset_exist

print('Found: '+str(len(task_exist))+' tasks')
print('Not found: '+str(len(task_not_exist))+' tasks')
print()
print('Found: '+str(len(dataset_exist))+' datasets')
print('Not found: '+str(len(dataset_not_exist))+' datasets')

dataset_exist = list(dataset_exist)

dataset_not_exist = list(dataset_not_exist)
dataset_not_exist = list(map(int, dataset_not_exist))

# Split dataset with meta-features into two datasets (train, test)
all_meta_features = pd.read_csv('C:\\Users\\Trogwald\\Desktop\\push_it\\AutoML\\meta_learning\\datasets_metafeatures\\study1_metafeatures.csv')

df_dids_for_train = all_meta_features.loc[all_meta_features['Dataset_ID'].isin(dataset_exist)].reset_index(drop=True)
df_dids_for_train.to_pickle('C:\\Users\\Trogwald\\Desktop\\push_it\\AutoML\\meta_learning\\datasets_metafeatures\\df_dids_for_train.pkl')

df_dids_for_test = all_meta_features.loc[all_meta_features['Dataset_ID'].isin(dataset_not_exist)].reset_index(drop=True)
df_dids_for_test.to_pickle('C:\\Users\\Trogwald\\Desktop\\push_it\\AutoML\\meta_learning\\datasets_metafeatures\\df_dids_for_test.pkl')

# Found: 1147 tasks
# Not found: 462 tasks
#
# Found: 492 datasets
# Not found: 20 datasets

In [6]:
import pandas as pd

path = r'C:\Users\Trogwald\Desktop\push_it\AutoML\meta_learning\datasets_metafeatures\\'

In [7]:
pd.read_pickle(path+'df_dids_for_test.pkl')

,Dataset_ID,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,c1,c2,...,tree_shape.mean,tree_shape.sd,var.mean,var.sd,var_importance.mean,var_importance.sd,vdb,w_lambda,worst_node.mean,worst_node.sd
0,1,0.146323,0.211739,0.594684,0.718035,0.097436,0.835897,0.013241,0.501116,0.651332,...,0.097392,0.140615,7.503905e+04,5.019701e+05,0.020408,0.049704,6.759387,9.789569e-09,0.782051,0.018131
1,21,0.035447,0.024917,1.756476,0.225072,0.085714,0.557143,0.125085,0.625003,0.435842,...,0.133185,0.123415,1.943711e-01,3.351337e-02,0.062500,0.061533,3.333819,1.520450e-01,0.600000,0.112687
2,33,0.094721,0.146011,1.939635,1.451148,0.097436,0.687179,0.058968,0.976635,0.062420,...,0.063186,0.107013,1.822471e+05,3.567739e+06,0.002597,0.012949,25.327024,1.542899e-02,0.594872,0.016217
3,173,0.048322,0.061941,1.066713,0.718662,0.092308,0.961538,0.040542,0.235193,0.920128,...,0.193115,0.157842,1.306566e-01,8.001143e-02,0.037037,0.060900,3.212116,7.115824e-01,0.961538,0.040542
4,174,0.095927,0.146472,1.125760,0.786316,0.092308,0.984615,0.032434,0.082059,0.976568,...,0.322917,0.158935,1.127982e-01,8.462555e-02,0.031250,0.099145,0.822880,4.743425e-09,0.984615,0.032434
5,292,0.048602,0.112406,1.453460,0.743584,0.093333,NaN,NaN,0.953197,0.120615,...,NaN,NaN,3.604349e-01,3.876424e-01,NaN,NaN,2.279478,3.670578e-01,NaN,NaN
6,314,0.056228,0.135874,0.781595,0.992375,0.096667,0.923333,0.016102,0.390324,0.835072,...,0.263021,0.134219,2.046849e+02,6.139778e+02,0.031250,0.098243,4.249827,7.061180e-01,0.923333,0.016102
7,345,0.205794,0.220456,1.127917,0.668793,0.578947,0.925000,0.064550,0.301210,0.872328,...,0.375000,0.125000,1.111382e-01,8.546541e-02,0.008130,0.046636,2.605408,2.958228e-31,0.925000,0.064550
8,449,0.056272,0.148770,1.728665,0.813161,0.159509,0.608088,0.042033,0.555043,0.419845,...,0.078619,0.128836,1.690356e+02,6.578257e+02,0.019608,0.034621,8.430600,3.117793e-11,0.644485,0.052683
9,460,0.063159,0.033059,1.675431,0.323976,0.087500,0.512500,0.181142,0.919732,0.088554,...,0.095877,0.097970,1.446562e-01,5.792059e-02,0.045455,0.032916,4.971468,1.608518e-01,0.387500,0.039528


In [8]:
pd.read_pickle(path+'df_dids_for_train.pkl')

,Dataset_ID,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,c1,c2,...,tree_shape.mean,tree_shape.sd,var.mean,var.sd,var_importance.mean,var_importance.sd,vdb,w_lambda,worst_node.mean,worst_node.sd
0,2,0.146323,0.211739,0.594684,7.180354e-01,0.097436,0.835897,0.013241,0.501116,0.651332,...,0.097392,0.140615,7.503905e+04,5.019701e+05,0.020408,0.049704,6.759387,9.789569e-09,0.782051,0.018131
1,3,0.063681,0.145646,0.566778,3.518098e-01,0.097297,0.629730,0.051042,0.997891,0.005827,...,0.098330,0.117758,1.326031e-01,8.426717e-02,0.027778,0.063705,5.607630,3.499563e-01,0.521622,0.052608
2,4,0.080232,0.092995,1.424929,4.091965e-01,0.280702,0.840000,0.125511,0.934849,0.163369,...,0.281250,0.178609,6.518634e+00,2.632104e+01,0.033333,0.091373,3.008230,1.748755e-01,0.596667,0.107094
3,5,0.061211,0.146104,1.381685,1.266458e+00,0.617257,0.597101,0.033827,0.648946,0.353360,...,0.054380,0.078759,1.639804e+02,3.905662e+02,0.003690,0.012834,3.979135,6.281995e-10,0.544251,0.010484
4,6,0.054407,0.060078,2.121566,2.392388e-01,0.094118,0.123529,0.043404,0.982120,0.004984,...,0.050000,0.086874,5.552415e+00,2.094882e+00,0.062500,0.045930,2.805844,2.098956e-04,0.094118,0.041130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,1116,0.119479,0.053265,3.110805,2.861727e-01,0.167000,0.890000,0.017638,0.602282,0.665275,...,0.098906,0.143024,5.363676e+03,4.715700e+03,0.004115,0.020404,4.823410,2.886580e-15,0.853000,0.004830
488,1117,0.094258,0.109173,1.912920,1.563455e-01,0.135802,0.705556,0.172580,0.857818,0.211174,...,0.206414,0.098040,1.783742e+06,5.884292e+06,0.090909,0.121541,7.421870,4.300071e-01,0.543056,0.083449
489,1119,0.071253,0.121629,1.699977,9.431405e-01,0.093333,0.700000,0.047140,0.855451,0.324397,...,0.142216,0.119599,1.777990e+08,1.408437e+09,0.015385,0.046013,3.376386,4.078779e-01,0.720000,0.028109
490,1120,0.157807,0.127196,1.999762,2.340556e-16,0.090909,0.763636,0.106708,0.945660,0.138462,...,0.201823,0.130708,1.363715e+03,1.788431e+03,0.100000,0.105464,4.510724,6.370966e-01,0.654545,0.083540


In [1]:
from AutoML.meta_learning.EvalParserAdaBoost import get_params_adaboost
from AutoML.meta_learning.util.utils import get_nearest_dids
import openml
import numpy as np
import pandas as pd
import pickle
import json

In [2]:
# Test data (20 samples from OpenML)
dqt = r'C:\Users\Trogwald\Desktop\push_it\AutoML\meta_learning\datasets_metafeatures\df_dids_for_test.pkl'
datasets_qualities_test = pd.read_pickle(dqt)
X_test = datasets_qualities_test.drop('Dataset_ID', axis=1).values
metafeatures = X_test[1]

n = 10
p_type = 'all'  # options 'automl, 'all'
closest_dids = get_nearest_dids(metafeatures)

df_hps = get_params_adaboost(2, 2, 10)
df_hps

,run_id,task_id,setup_id,flow_id,flow_name,data_id,data_name,function,upload_time,uploader,uploader_name,value,values,array_data,parameters
0,10160806,2,8086270,9689,sklearn.pipeline.Pipeline(imputer=sklearn.prep...,2,anneal,predictive_accuracy,2019-04-13 06:09:08,8420,T I,0.991091,None,None,{'sklearn.pipeline.Pipeline(imputer=sklearn.pr...
1,1860456,2,36565,5909,sklearn.pipeline.Pipeline(imputer=sklearn.prep...,2,anneal,predictive_accuracy,2017-03-27 22:32:43,1022,Randal Olson,0.983296,None,None,{'sklearn.ensemble.forest.RandomForestClassifi...
2,5950015,2,3984760,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-14 15:39:47,1,Jan van Rijn,0.963252,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...
3,5950070,2,3984815,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-14 15:43:34,1,Jan van Rijn,0.962138,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...
4,4855681,2,2891090,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-08 06:43:16,1,Jan van Rijn,0.961024,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3601,5930403,2,3965152,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-14 01:15:43,1,Jan van Rijn,0.685969,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...
3602,5956713,2,3991454,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-14 23:11:06,1,Jan van Rijn,0.685969,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...
3603,6043409,2,4078067,6952,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-24 23:10:11,1,Jan van Rijn,0.672606,None,None,{'openmlstudy14.preprocessing.ConditionalImput...
3604,4970024,2,3005421,6970,sklearn.pipeline.Pipeline(imputation=openmlstu...,2,anneal,predictive_accuracy,2017-07-08 19:09:37,1,Jan van Rijn,0.669265,None,None,{'sklearn.tree.tree.DecisionTreeClassifier(10)...


In [13]:
mydict = df_hps['parameters'].values[318]

foodict = {k: v for k, v in mydict.items() if k.startswith('sklearn.ensemble.weight_boosting.AdaBoostClassifier')}
foodict

{'sklearn.ensemble.weight_boosting.AdaBoostClassifier(base_estimator=sklearn.tree.tree.DecisionTreeClassifier)(2)_algorithm': '"SAMME"',
 'sklearn.ensemble.weight_boosting.AdaBoostClassifier(base_estimator=sklearn.tree.tree.DecisionTreeClassifier)(2)_learning_rate': '0.6933486293410185',
 'sklearn.ensemble.weight_boosting.AdaBoostClassifier(base_estimator=sklearn.tree.tree.DecisionTreeClassifier)(2)_n_estimators': '262',
 'sklearn.ensemble.weight_boosting.AdaBoostClassifier(base_estimator=sklearn.tree.tree.DecisionTreeClassifier)(2)_random_state': '14888'}

In [15]:
df_hps.values[318][2]

{'sklearn.tree.tree.DecisionTreeClassifier(10)_class_weight': 'null',
 'sklearn.tree.tree.DecisionTreeClassifier(10)_criterion': '"gini"',
 'sklearn.tree.tree.DecisionTreeClassifier(10)_max_depth': '4',
 'sklearn.tree.tree.DecisionTreeClassifier(10)_max_features': 'null',
 'sklearn.tree.tree.DecisionTreeClassifier(10)_max_leaf_nodes': 'null',
 'sklearn.tree.tree.DecisionTreeClassifier(10)_min_impurity_split': '1e-07',
 'sklearn.tree.tree.DecisionTreeClassifier(10)_min_samples_leaf': '1',
 'sklearn.tree.tree.DecisionTreeClassifier(10)_min_samples_split': '2',
 'sklearn.tree.tree.DecisionTreeClassifier(10)_min_weight_fraction_leaf': '0.0',
 'sklearn.tree.tree.DecisionTreeClassifier(10)_presort': 'false',
 'sklearn.tree.tree.DecisionTreeClassifier(10)_random_state': '52408',
 'sklearn.tree.tree.DecisionTreeClassifier(10)_splitter': '"best"',
 'openmlstudy14.preprocessing.ConditionalImputer(2)_axis': '0',
 'openmlstudy14.preprocessing.ConditionalImputer(2)_categorical_features': '[0, 1, 2,